In [306]:
import requests
import pandas as pd

In [307]:
api_key = 'MDk1N2E0MzctNDY3Ni00NDIzLWJlZjMtMzM3YTU5NzkzNTQ2'
workspace_id = '63ada8fc28d7584d0c563d3f'


In [308]:
reporturl = 'https://reports.api.clockify.me/v1'

In [309]:
url = 'https://api.clockify.me/api/v1'
workspaceId = '63ada8fc28d7584d0c563d3f'
key = 'MDk1N2E0MzctNDY3Ni00NDIzLWJlZjMtMzM3YTU5NzkzNTQ2'

In [310]:
# get usernames 
r = requests.get(url+f'/workspaces/{workspaceId}/users',headers={'X-Api-Key':key})
#list of users
usernames = r.json()
#key = user id, value = user name 
user_lookup = dict()
for row in usernames:
    user_lookup[row['id']] = row['name']


In [311]:
for u in usernames:
    if u['name']=='Kieran Clarke':
        kid = u['id']
kid

'638f6e0fd9c69c33cb80d627'

In [312]:
# get projects
r = requests.get(url+f'/workspaces/{workspaceId}/projects',headers={'X-Api-Key':key})
#list of projects
projects = r.json()
#key = proj id, value = proj name 
proj_lookup = dict()
for row in projects:
    proj_lookup[row['id']] = row['name']

In [313]:
# get tags
r = requests.get(url+f'/workspaces/{workspaceId}/tags',headers={'X-Api-Key':key})
#list of tags
tags = r.json()
#key = tag id, value = tag name 
tag_lookup = dict()
for row in tags:
    tag_lookup[row['id']] = row['name']

In [314]:
# get who is in the group.    
r = requests.get(url+f'/workspaces/{workspaceId}/user-groups',headers={'X-Api-Key':key})
#list of groups
j = r.json()
# get ds user ids:
for g in j:
    if g['name'] == 'Data Science':
        users = g['userIds']
users.append(kid)

In [315]:
#get task from project id

In [316]:

from collections import defaultdict
entries = defaultdict(list)

for userId in users:
    page_count = 1
    while True:
        r = requests.get(url+f'/workspaces/{workspaceId}/user/{userId}/time-entries',headers={'X-Api-Key':key},params={'page-size':5000,'page':page_count})
        entries[user_lookup[userId]] += r.json()
        if len(r.json()) < 5000:
            break
        
        page_count+=1
 # entries contains list of time entries for each user in ds 

In [317]:
entries

defaultdict(list,
            {'Ellie Biessek': [{'id': '63bea77e6e59b4047b652a3b',
               'description': '',
               'tagIds': [],
               'userId': '6290b4a1b04dbc3657c0fe5e',
               'billable': False,
               'taskId': None,
               'projectId': '63b42cf2abb1846bc7982697',
               'timeInterval': {'start': '2023-01-11T12:11:42Z',
                'end': '2023-01-11T15:49:42Z',
                'duration': 'PT3H38M'},
               'workspaceId': '63ada8fc28d7584d0c563d3f',
               'isLocked': False,
               'customFieldValues': [],
               'type': 'REGULAR',
               'kioskId': None},
              {'id': '63be687a5e398c3868752dee',
               'description': '',
               'tagIds': None,
               'userId': '6290b4a1b04dbc3657c0fe5e',
               'billable': False,
               'taskId': None,
               'projectId': '63b42cf2abb1846bc7982697',
               'timeInterval': {'start':

In [318]:
for user, time_entries in entries.items():
    for time_entry in time_entries:
        tag_id = time_entry['tagIds'][0] if time_entry['tagIds'] and len(time_entry['tagIds'])>0 else None
        proj_id = time_entry['projectId']
        time_entry['tag'] = tag_lookup.get(tag_id, None)
        time_entry['project']= proj_lookup.get(proj_id, None)


In [319]:
df= pd.DataFrame([(user, entry['id'], entry['project'], entry['projectId'],entry['description'], entry['tag'], entry['tagIds'],  entry['timeInterval'])
for user, entries in entries.items() for entry in entries], columns = ['user', 'id', 'project', 'project_id', 'description', 'tag', 'tag_id', 'time_interval'])


In [320]:
df['user'].unique()

array(['Ellie Biessek', 'Toby Crisford', 'Ella Zheng', 'Matt Pang',
       'Georgi Pavlovski', 'Ben Collins', 'Emma Hunt', 'Talia Solel',
       'Kieran Clarke'], dtype=object)

In [321]:
df['start_time'] = df['time_interval'].apply(lambda x: (x['start']))
df['end__time'] = df['time_interval'].apply(lambda x: (x['end']))
df.drop('time_interval', axis= 1 , inplace=True)

In [361]:
projects_cats = { 

'indirect' : [ 'Meetings', 'Admin', 'Training (Corporate)' ,'Training (Other)', 'Recruitment'], 

'time_off' : ['Holiday' , 'Bank Holiday', 'Sick Leave', 'Medical Appointment',], 
}

In [362]:

def categorise_project(project, projects_cats):
    for cat, proj_list in projects_cats.items():
        if project in proj_list:
            return cat
    return 'direct'

In [356]:
categorise_project('Sick Leave', projects_cats)

'time_off'

In [363]:
df['Category'] = df['project'].apply(lambda x: categorise_project(x, projects_cats))


In [326]:
from datetime import datetime
df['start_time']=df['start_time'].apply(lambda x : datetime.fromisoformat(x[:-1]))
df['end__time']=df['end__time'].apply(lambda x : datetime.fromisoformat(x[:-1]))



In [327]:
df=df[df['start_time'].dt.year>2022]

In [328]:
df['duration'] = df.apply(lambda x: (x['end__time'] - x['start_time']).total_seconds()/3600, axis=1)

In [358]:
def utilisation(month):
    df_gb_user_cats = pd.DataFrame(df[df['start_time'].dt.month==month].groupby(['user', 'Category'])['duration'].sum()).unstack().fillna(0)
    df_gb_user_cats.columns = df_gb_user_cats.columns.get_level_values(1)
    df_gb_user_cats.reset_index(inplace=True)
    totals = df_gb_user_cats.sum(numeric_only=True, axis=0).tolist()
    totals.insert(0, 'Total')
    new_row = pd.DataFrame([totals], columns=df_gb_user_cats.columns)
    df_gb_user_cats = df_gb_user_cats.append(new_row, ignore_index=True)
    num_cols = df_gb_user_cats.select_dtypes(include=['int', 'float']).columns
    df_gb_user_cats['total_hours'] = df_gb_user_cats[num_cols].sum(axis=1)
    df_gb_user_cats['utilisation %'] = df_gb_user_cats['direct']*100/(df_gb_user_cats['indirect']+ df_gb_user_cats['direct'])

    return df_gb_user_cats


In [364]:
utilisation_feb= utilisation(2)

C:\Users\emma\AppData\Local\Temp\ipykernel_25980\668336845.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gb_user_cats = df_gb_user_cats.append(new_row, ignore_index=True)


In [365]:
utilisation_feb.to_excel('utilisation_feb.xlsx', index=False)

In [332]:
utilisation(1)

C:\Users\emma\AppData\Local\Temp\ipykernel_25980\668336845.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gb_user_cats = df_gb_user_cats.append(new_row, ignore_index=True)


Category,user,direct,indirect,time_off,total_hours,utilisation %
0,Ben Collins,125.750000,10.250000,42.500000,178.500000,92.463235
1,Ella Zheng,117.500000,38.000000,10.000000,165.500000,75.562701
2,Ellie Biessek,25.183333,0.000000,32.000000,57.183333,100.000000
3,Emma Hunt,14.000000,0.000000,0.000000,14.000000,100.000000
4,Georgi Pavlovski,122.000000,44.466667,9.500000,175.966667,73.287946
5,Matt Pang,65.750000,60.000000,46.500000,172.250000,52.286282
6,Talia Solel,146.000000,21.000000,23.000000,190.000000,87.425150
7,Toby Crisford,137.083333,26.916667,10.666667,174.666667,83.587398
8,Total,753.266667,200.633333,174.166667,1128.066667,78.967048


In [333]:
df_gb_user_cats_jan = pd.DataFrame(df[df['start_time'].dt.month==1].groupby(['user', 'Category'])['duration'].sum()).unstack().fillna(0)
df_gb_user_cats_jan.columns = df_gb_user_cats_jan.columns.get_level_values(1) # use the second level of the column index as the column names
df_gb_user_cats_jan = df_gb_user_cats_jan.reset_index()

In [334]:
totals = df_gb_user_cats_jan.sum(numeric_only=True, axis=0).tolist()
totals.insert(0, 'Total')
new_row = pd.DataFrame([totals], columns=df_gb_user_cats_jan.columns)
df_gb_user_cats_jan = df_gb_user_cats_jan.append(new_row, ignore_index=True)

C:\Users\emma\AppData\Local\Temp\ipykernel_25980\3276887719.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gb_user_cats_jan = df_gb_user_cats_jan.append(new_row, ignore_index=True)


In [335]:
df_gb_user_cats_jan

Category,user,direct,indirect,time_off
0,Ben Collins,125.750000,10.250000,42.500000
1,Ella Zheng,117.500000,38.000000,10.000000
2,Ellie Biessek,25.183333,0.000000,32.000000
3,Emma Hunt,14.000000,0.000000,0.000000
4,Georgi Pavlovski,122.000000,44.466667,9.500000
5,Matt Pang,65.750000,60.000000,46.500000
6,Talia Solel,146.000000,21.000000,23.000000
7,Toby Crisford,137.083333,26.916667,10.666667
8,Total,753.266667,200.633333,174.166667


In [336]:
# df_gb_user_cats_jan['utilisation %'] = df_gb_user_cats_jan['direct']*100/(df_gb_user_cats_jan['indirect']+df_gb_user_cats_jan['unknown']+ df_gb_user_cats_jan['direct'])
# df_gb_user_cats_jan.rename(columns={'not_work':'time_off' }, inplace=True)


In [337]:
# df_gb_user_cats_jan['total_hours']= df_gb_user_cats_jan['direct']+ df_gb_user_cats_jan['indirect'] + df_gb_user_cats_jan['unknown'] + df_gb_user_cats_jan['time_off']

In [338]:
df_gb_user_cats_jan

Category,user,direct,indirect,time_off
0,Ben Collins,125.750000,10.250000,42.500000
1,Ella Zheng,117.500000,38.000000,10.000000
2,Ellie Biessek,25.183333,0.000000,32.000000
3,Emma Hunt,14.000000,0.000000,0.000000
4,Georgi Pavlovski,122.000000,44.466667,9.500000
5,Matt Pang,65.750000,60.000000,46.500000
6,Talia Solel,146.000000,21.000000,23.000000
7,Toby Crisford,137.083333,26.916667,10.666667
8,Total,753.266667,200.633333,174.166667


In [339]:
df_gb_user_cats_jan.to_excel('utilisation_jan.xlsx', index=False)

In [340]:
df['week_no']= df['start_time'].apply(lambda x: x.isocalendar()[1])



In [341]:
from datetime import timedelta
df['w/c']= df['start_time'].apply(lambda x:(x - timedelta(days=x.weekday())).strftime('%Y-%m-%d'))

In [342]:
df

,user,id,project,project_id,description,tag,tag_id,start_time,end__time,Category,duration,week_no,w/c
0,Ellie Biessek,63bea77e6e59b4047b652a3b,DS - Flight Scoring,63b42cf2abb1846bc7982697,,None,[],2023-01-11 12:11:42,2023-01-11 15:49:42,direct,3.633333,2,2023-01-09
1,Ellie Biessek,63be687a5e398c3868752dee,DS - Flight Scoring,63b42cf2abb1846bc7982697,,None,None,2023-01-11 07:42:50,2023-01-11 11:36:50,direct,3.900000,2,2023-01-09
2,Ellie Biessek,63bd27d16667c75020e16726,DS - Flight Scoring,63b42cf2abb1846bc7982697,,None,None,2023-01-10 09:00:44,2023-01-10 17:11:44,direct,8.183333,2,2023-01-09
3,Ellie Biessek,63bbd6b26e59b4047b22f57a,None,None,,None,[],2023-01-09 08:56:18,2023-01-09 10:23:18,direct,1.450000,2,2023-01-09
4,Ellie Biessek,63b7e58236fb0f5ed84c130b,None,None,,None,None,2023-01-06 09:00:00,2023-01-06 17:01:00,direct,8.016667,1,2023-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,Kieran Clarke,63fc8bcce99cfa413db8f351,DS - Miscellenous,63b42d3babb1846bc7982a13,brake temperature model. \nover week added jal...,Development,"[63adb5ee22002800627416ff, 63adb61222002800627...",2023-02-22 09:00:00,2023-02-22 17:00:00,direct,8.000000,8,2023-02-20
1024,Kieran Clarke,63fc8bc66a6acd412554e150,DS - Miscellenous,63b42d3babb1846bc7982a13,brake temperature model. \nover week added jal...,Development,"[63adb5ee22002800627416ff, 63adb61222002800627...",2023-02-21 09:15:00,2023-02-21 17:00:00,direct,7.750000,8,2023-02-20
1025,Kieran Clarke,63fc8bb7e99cfa413db8f0ec,DS - Miscellenous,63b42d3babb1846bc7982a13,brake temperature model. \nover week added jal...,Development,"[63adb5ee22002800627416ff, 63adb61222002800627...",2023-02-20 14:00:00,2023-02-20 17:00:00,direct,3.000000,8,2023-02-20
1026,Kieran Clarke,63fc8b345092657bd7db79f9,Meetings,63adaec628d7584d0c56b7ff,,Meetings,[63adb60385786a45814e5701],2023-02-20 13:00:00,2023-02-20 14:00:00,indirect,1.000000,8,2023-02-20


In [343]:
weekly_hrs_byuser =pd.DataFrame(df[df['start_time']< '2023-02-27 00:00:00'].groupby(['user','week_no'])['duration'].sum().unstack())
weekly_hrs_byuser

week_no,1,2,3,4,5,6,7,8
user,,,,,,,,
Ben Collins,42.500000,40.000000,40.000000,40.000000,40.000000,40.250000,40.50,40.00
Ella Zheng,38.000000,37.500000,37.500000,37.500000,37.500000,42.000000,42.50,42.50
Ellie Biessek,40.016667,17.166667,NaN,NaN,NaN,NaN,NaN,NaN
Emma Hunt,NaN,NaN,NaN,NaN,38.000000,40.000000,40.00,40.00
Georgi Pavlovski,40.000000,40.000000,39.966667,40.000000,40.000000,40.000000,40.00,40.00
Kieran Clarke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.75
Matt Pang,37.250000,36.500000,37.500000,44.000000,40.466667,40.750000,39.00,42.00
Talia Solel,41.500000,42.250000,40.750000,42.000000,48.000000,43.750000,41.75,40.00
Toby Crisford,39.750000,39.583333,38.916667,40.083333,40.000000,40.166667,40.50,40.75


In [344]:

weekly_hrs_byuser['average'] = weekly_hrs_byuser.mean(axis=1)


In [345]:
weekly_hrs_byuser

week_no,1,2,3,4,5,6,7,8,average
user,,,,,,,,,
Ben Collins,42.500000,40.000000,40.000000,40.000000,40.000000,40.250000,40.50,40.00,40.406250
Ella Zheng,38.000000,37.500000,37.500000,37.500000,37.500000,42.000000,42.50,42.50,39.375000
Ellie Biessek,40.016667,17.166667,NaN,NaN,NaN,NaN,NaN,NaN,28.591667
Emma Hunt,NaN,NaN,NaN,NaN,38.000000,40.000000,40.00,40.00,39.500000
Georgi Pavlovski,40.000000,40.000000,39.966667,40.000000,40.000000,40.000000,40.00,40.00,39.995833
Kieran Clarke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.75,38.750000
Matt Pang,37.250000,36.500000,37.500000,44.000000,40.466667,40.750000,39.00,42.00,39.683333
Talia Solel,41.500000,42.250000,40.750000,42.000000,48.000000,43.750000,41.75,40.00,42.500000
Toby Crisford,39.750000,39.583333,38.916667,40.083333,40.000000,40.166667,40.50,40.75,39.968750


In [346]:
df_gb_user_cats_weekly= pd.DataFrame(df.groupby(['user', 'Category', 'week_no', 'w/c'])['duration'].sum())


weekly_utilisation_by_user = df_gb_user_cats_weekly.pivot_table(values='duration', index=['user', 'week_no', 'w/c'], columns='Category', aggfunc='sum').reset_index().fillna(0)


weekly_utilisation_by_user['total_hours']= weekly_utilisation_by_user['direct'] +weekly_utilisation_by_user['indirect'] + weekly_utilisation_by_user['time_off'] 
weekly_utilisation_by_user

weekly_utilisation_by_user['utilisation %'] = weekly_utilisation_by_user['direct']*100/(weekly_utilisation_by_user['indirect']+ weekly_utilisation_by_user['direct'])

weekly_utilisation_by_user.to_excel('weekly_utilisation.xlsx', index=False)



In [347]:
weekly_utilisation_by_user[weekly_utilisation_by_user['user']=='Emma Hunt']

Category,user,week_no,w/c,direct,indirect,time_off,total_hours,utilisation %
19,Emma Hunt,5,2023-01-30,38.000000,0.000000,0.0,38.0,100.000000
20,Emma Hunt,6,2023-02-06,40.000000,0.000000,0.0,40.0,100.000000
21,Emma Hunt,7,2023-02-13,24.000000,0.000000,16.0,40.0,100.000000
22,Emma Hunt,8,2023-02-20,24.000000,0.000000,16.0,40.0,100.000000
23,Emma Hunt,9,2023-02-27,17.083333,6.916667,0.0,24.0,71.180556
